In [17]:
import pandas as pd
from seml import get_results

In [ ]:
df = get_results(
    'dp_timeseries_dp_train_standard_eval_user_level',
    [
        'config.seed',
        'config.estimator_name',
        'config.dp_optimizer_kwargs.max_grad_norm',
        'config.dp_optimizer_kwargs.noise_multiplier',
        'config.estimator_kwargs.relative_context_length',
        'config.dp_accountant_kwargs.budget_epsilon',
        'config.neighboring_relation.size',
        'result.metrics_test.mean_wQuantileLoss',
        'result.metrics_test.MAPE',
    ],
    to_data_frame=True,
    filter_dict={
        'config.dataset_kwargs.dataset_name': 'electricity',
        'config.estimator_kwargs.trainer_kwargs.max_epochs': 8000,
        'config.top_level_mode': 'sampling_without_replacement',
        'config.instances_per_sequence': 1,
        'config.estimator_kwargs.batch_size': {'$in': [128]},
        'config.dp_optimizer_kwargs.max_grad_norm': {'$in': [0.001, 0.0001]},
        'config.dp_optimizer_kwargs.noise_multiplier': {'$in': [1.0, 2.0]},
        'config.estimator_kwargs.relative_context_length': {'$in': [1, 2]},
        'config.dp_accountant_kwargs.budget_delta': 1e-7,
        'config.estimator_kwargs.lags_seq': {'$in': [None,
                                                     [1, 2, 3, 4, 5, 6, 7, 23, 24, 25]]},
        'config.tight_privacy_loss': True
    }
)

In [19]:
columns = df.columns[1:]

df = df.rename(columns={
    c: c.split('.')[-1]
    for c in columns
})

In [ ]:
df

In [21]:
def create_table(df: pd.DataFrame,
                 relative_context_length: int,
                 max_grad_norm: float,
                 noise_multiplier: float,
                 budget_epsilon: float,
                 errors: bool = False
) -> pd.DataFrame:

    df = df.copy()

    df = df[
        (df['relative_context_length'] == relative_context_length)
        & (df['max_grad_norm'] == max_grad_norm)
        & (df['noise_multiplier'] == noise_multiplier)
        & (df['budget_epsilon'] == budget_epsilon)
    ]

    print(len(df))

    # Average over random seeds
    if not errors:
        df = df.groupby(
            ['max_grad_norm', 'noise_multiplier', 'relative_context_length', 'estimator_name', 'budget_epsilon', 'size']
        ).mean().reset_index()
    else:
        df = df.groupby(
            ['max_grad_norm', 'noise_multiplier', 'relative_context_length', 'estimator_name', 'budget_epsilon', 'size']
        ).std().reset_index()

    print(df.columns)

    df = df.drop(columns=[
        '_id',
        'seed', 'relative_context_length',
        'max_grad_norm', 'noise_multiplier', '_id', 'budget_epsilon', 'budget_epsilon'])

    df = df.sort_values(by=['estimator_name', 'size'])

    return df

### Relative context length 1, noise multiplier 2

In [ ]:
df_mean = create_table(df, 1, 0.0001, 2.0, 4.0)
df_mean

In [ ]:
df_std = create_table(df, 1, 0.0001, 2.0, 4.0, errors=True)
df_std

In [ ]:
i = 0
names = ['DLinear', 'DeepAR', 'iTransf.', 'SimpleFF']

acc = f'{names[0]} &'

for i, (mean, std) in enumerate(zip(df_mean['mean_wQuantileLoss'], df_std['mean_wQuantileLoss'])):

    if (i > 0) and ((i % 4)  == 0):
        acc += ' \\\\'
        print(acc)
        acc = f'{names[i // 4]} &'
    acc += ' $' + f'{mean:.3f}' + '$ '
    acc += '\\tiny{$\\pm ' + f'{std:.3f}' + '$} &'

acc += ' \\\\ '
print(acc)